# Face recognition model

### Load images and detect faces on them

In [ ]:
import cv2 
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from random import shuffle


DIRECTORY = r'C:\Users\Admin\Desktop\DATASET'

ARTISTS = [
    'Adele', 'Angelina Jolie', 'Arnold Schwarzenegger', 
    'Bon Jovi', 'Brad Pitt', 'Chuck Norris', 'Conor Mcgregor',
    'Cristiano Ronaldo', 'Ed Sheeran', 'Eddie Murphy'
    ]
    
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
IMAGE_SIZE = 160
train_data = []


for artist in ARTISTS:
    
    path = os.path.join(DIRECTORY, artist)
    class_label = ARTISTS.index(artist)
    
    for image in os.listdir(path):
        
        img = cv2.imread(os.path.join(path, image), cv2.IMREAD_COLOR)
        detected_faces = face_cascade.detectMultiScale(img, 1.3, 5)
        
        try:
            (x,y,w,h) = detected_faces[0]
            roi = img[y:y+h, x:x+w]
            new_array = cv2.resize(roi, (IMAGE_SIZE, IMAGE_SIZE))
            train_data.append([new_array, class_label]) 
        except:
            print(f'Face not found: {path} {image}')
            

### Shuffle images and prepare to be saved

In [ ]:
shuffle(train_data)

X = []
y = []

for image, label in train_data:
    X.append(image)
    y.append(label)
    
X = np.array(X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
y = np.array(y)

### Save training data

In [ ]:
with open('faces_x_pickle.pkl', 'wb') as pickle_file:
    pickle.dump(X, pickle_file)
    
with open('faces_y_pickle.pkl', 'wb') as pickle_file:
    pickle.dump(y, pickle_file)

### Load data from file

In [ ]:
import pickle 

with open('faces_x_pickle.pkl', 'rb') as pickle_file:
    X = pickle.load(pickle_file)
    
with open('faces_y_pickle.pkl', 'rb') as pickle_file:
    y = pickle.load(pickle_file)

### Normalize data to be in range [0, 1]

In [ ]:
X = X/255.0

## CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

NAME = f"Facial-recognition-celebs-best-best-{int(time.time())}"
tensorboard = TensorBoard(log_dir=f'logs3\\{NAME}')

model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=X.shape[1:]),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'), 
    MaxPooling2D(2,2),
    
    Conv2D(256, (3,3), activation='relu'), 
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.05),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', 
             optimizer='adam',
             metrics=['accuracy'])


model.fit(X, y, batch_size=16, epochs=20, validation_split=0.15, callbacks=[tensorboard])

model.save('best_saved_model/')

## Additional - during research

### Build. train and save logs in tensorboard of many models to choose the best architecture of CNN

In [ ]:
import time

dense_layers = [0, 1]
layer_filters = [32, 64, 128, 256]
conv_layers = [2, 3, 4]

for dense_layer in dense_layers:
    for layer_filter in layer_filters:
        for conv_layer in conv_layers:
            NAME = f'{conv_layer}-conv-{layer_filter}-filters-{dense_layer}-dense-{int(time.time())}'
            tensorboard = TensorBoard(log_dir=f'logs\\{NAME}')
            model = Sequential()
            model.add(Conv2D(layer_filter, (3,3), activation='relu', input_shape=X.shape[1:]))
            model.add(MaxPooling2D(2,2))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_filter, (3,3), activation='relu'))
                model.add(MaxPooling2D(2,2))
                
            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(1024, activation='relu'))
            
            model.add(Dense(10, activation='softmax'))
            model.compile(loss='sparse_categorical_crossentropy', 
                         optimizer='adam',
                         metrics=['accuracy'])
            model.fit(X, y, batch_size=16, epochs=15, validation_split=0.15, callbacks=[tensorboard])